In [1]:
import sys
sys.path.append('/home/yeyiqi/Documents/repos/GaussianAvatars')
sys.path.append('/home/yeyiqi/.conda/envs/gaussian-avatars/lib/python3.10/site-packages')

In [2]:
from plyfile import PlyData, PlyElement
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import sys
import os
from pathlib import Path
import pickle
from typing import NamedTuple, Optional
from utils.graphics_utils import ClothMesh, SmplxMesh, ClothedSmplxMesh, read_ply_mesh
from utils.loss_utils import l1_loss, ssim
from scene.cloth_smplx_model import ClothSmplxGaussianModel
from scene.dataset_readers import getNerfppNorm
from gaussian_renderer import render
from tqdm import tqdm
from utils.graphics_utils import getWorld2View2, getProjectionMatrix

In [3]:
class CustomArgs(NamedTuple):
    upper_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Semantic/clothes/cloth-f00011_upper.ply'
    outer_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Semantic/clothes/cloth-f00011_outer.ply'
    lower_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Semantic/clothes/cloth-f00011_lower.ply'
    shoe_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Semantic/clothes/cloth-f00011_shoe.ply'
    smplx_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/SMPLX/mesh-f00011_smplx.ply'
    cam_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Capture/cameras.pkl'
    frame = '00011'
    model_path = '/home/yeyiqi/Documents/WD/00122/Outer/Take9/Capture'
    resolution = 1
    iterations = 600_000  # 30_000 (original)
    position_lr_init = 0.005  # (scaled up according to mean triangle scale)  #0.00016 (original)
    position_lr_final = 0.00005  # (scaled up according to mean triangle scale) # 0.0000016 (original)
    position_lr_delay_mult = 0.01
    position_lr_max_steps = 600_000  # 30_000 (original)
    feature_lr = 0.0025
    opacity_lr = 0.05
    scaling_lr = 0.017  # (scaled up according to mean triangle scale)  # 0.005 (original)
    rotation_lr = 0.001
    densification_interval = 2_000  # 100 (original)
    opacity_reset_interval = 60_000 # 3000 (original)
    densify_from_iter = 10_000  # 500 (original)
    densify_until_iter = 600_000  # 15_000 (original)
    densify_grad_threshold = 0.0002
    flame_expr_lr = 1e-3
    flame_trans_lr = 1e-6
    flame_pose_lr = 1e-5
    percent_dense = 0.01
    lambda_dssim = 0.2
    lambda_xyz = 1e-2
    threshold_xyz = 1.
    metric_xyz = False
    lambda_scale = 1.
    threshold_scale = 0.6
    metric_scale = False
    lambda_dynamic_offset = 0.
    lambda_laplacian = 0.
    lambda_dynamic_offset_std = 0  #1.
    # pipe parameters
    convert_SHs_python = False
    compute_cov3D_python = False
    debug = False
    data_device = 'cuda'

In [4]:
args = CustomArgs()
upper_mesh = read_ply_mesh(args.upper_path)
outer_mesh = read_ply_mesh(args.outer_path)
lower_mesh = read_ply_mesh(args.lower_path)
shoe_mesh = read_ply_mesh(args.shoe_path)
smplx_mesh = read_ply_mesh(args.smplx_path, type='smplx')


In [5]:
meshes = [outer_mesh, upper_mesh, lower_mesh, shoe_mesh, smplx_mesh]